In [1]:
!pip install rasterio geopandas ultralytics

In [2]:
import cv2
import time
import yaml
import json
import torch
import optuna
import pickle
import logging
import numpy as np
import pandas as pd
import seaborn as sns
from typing import Dict

import albumentations as A
import matplotlib.pyplot as plt

from pathlib import Path
from ultralytics import YOLO
from collections import defaultdict
from albumentations.pytorch import ToTensorV2
from typing import Dict, List, Tuple, Optional

In [4]:
def get_kaggle_gpu_info():
    """Check available GPU resources in Kaggle"""
    if torch.cuda.is_available():
        gpu_name = torch.cuda.get_device_name(0)
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"GPU: {gpu_name}")
        print(f"GPU Memory: {gpu_memory:.1f} GB")
        return gpu_memory
    else:
        print("No GPU available, using CPU")
        return 0

In [6]:
def train_progressive_augmentation(model, project, name):
    """
    Progressive training with different augmentation intensities
    Good for getting the best performance in Kaggle's time constraints
    """
    
    gpu_memory = get_kaggle_gpu_info()
    
    # Base configuration
    base_config = {
        'data': '/kaggle/input/tree-detection/tree_detection/data.yaml',
        'imgsz': 640 if gpu_memory >= 12 else 512,
        'batch': 8 if gpu_memory >= 15 else (6 if gpu_memory >= 12 else 4),
        'task': 'detect',
        'project': project,
        'optimizer': 'AdamW',
        'amp': True,
        'device': 0,
        'workers': 2,
        'cache': False,
        'cos_lr': True,
        'val': True,
        'plots': True,
        'save': True,
        'verbose': True,
        'seed': 42,
        'exist_ok': True,
        'pretrained': True,
        'patience': 0,
    }
    
    # Phase 1: Heavy augmentation (first 60 epochs)
    print("\n🚀 Phase 1: Heavy Augmentation Training (60 epochs)")
    phase1_config = base_config.copy()
    phase1_config.update({
        'name': f'{name}_phase1',
        'epochs': 60,
        'lr0': 0.001,
        'mosaic': 1.0,
        'mixup': 0.3,
        'copy_paste': 0.4,
        'degrees': 15.0,
        'translate': 0.2,
        'scale': 0.6,
        'hsv_s': 0.8,
        'hsv_v': 0.5,
        'erasing': 0.5,
        'close_mosaic': 10,
    })
    
    results_phase1 = model.train(**phase1_config)
    
    # Phase 2: Fine-tuning with lighter augmentation (40 epochs)
    print("\n🎯 Phase 2: Fine-tuning with Light Augmentation (40 epochs)")
    phase2_config = base_config.copy()
    phase2_config.update({
        'name': f'{name}_phase2',
        'epochs': 40,
        'lr0': 0.0001,  # Lower learning rate for fine-tuning
        'mosaic': 0.3,  # Reduced mosaic
        'mixup': 0.1,   # Minimal mixup
        'copy_paste': 0.1,
        'degrees': 5.0,  # Reduced rotation
        'translate': 0.05,
        'scale': 0.3,
        'hsv_s': 0.4,
        'hsv_v': 0.3,
        'erasing': 0.2,
        'resume': True,  # Resume from phase 1
        'close_mosaic': 5,
    })
    
    results_phase2 = model.train(**phase2_config)
    
    print("✅ Progressive training completed!")
    return results_phase2

In [7]:
# Usage examples:
def main():
    """Example usage in Kaggle"""
    
    # Load your model
    model = YOLO('yolo11l.pt')  # or your specific model
    project = 'tree_detection'
    name = 'augmented_run'
    
    print("Choose training method:")
    print("1. Standard training with heavy augmentation")
    print("2. Progressive training (recommended for best results)")
    
    # Option 1: Standard training with comprehensive augmentation
    results = train_with_augmentation(model, project, name)
    
    # Option 2: Progressive training (uncomment to use instead)
    # results = train_progressive_augmentation(model, project, name)
    
    # After training, you can evaluate or make predictions
    print(f"Training completed! Best model saved in: {project}/{name}/weights/best.pt")
    
    return results

if __name__ == "__main__":
    main()

Choose training method:
1. Standard training with heavy augmentation
2. Progressive training (recommended for best results)
GPU: Tesla P100-PCIE-16GB
GPU Memory: 17.1 GB
🌳 Starting Kaggle-Optimized Tree Detection Training
📋 Augmentation Configuration:
  - Batch Size: 8
  - Image Size: 640
  - Mosaic: 1.0
  - MixUp: 0.2
  - Copy-Paste: 0.3
  - HSV Augmentation: H=0.015, S=0.7, V=0.4
  - Geometric: Degrees=10.0, Scale=0.5
  - Random Erasing: 0.4
  - AMP Enabled: True
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in the future.
WARNING ⚠️ 'crop_fraction' is deprecated and will be removed in the future.
WARNING ⚠️ 'save_hybrid' is deprecated and will be removed in the future.
Ultralytics 8.3.202 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=0.25, copy_

/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
/usr/local/lib/python3.11/dist-packages/matplotlib/colors.py:721: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


                   all        223      13063      0.926      0.929       0.94      0.608
Speed: 0.4ms preprocess, 36.2ms inference, 0.0ms loss, 0.8ms postprocess per image
Saving /kaggle/working/tree_detection/augmented_run/predictions.json...
Results saved to /kaggle/working/tree_detection/augmented_run
✅ Training completed successfully!
Training completed! Best model saved in: tree_detection/augmented_run/weights/best.pt
